# SparkProject Movie Recommender Systems 

### To Paytm team:

Please make sure that the following file dependence is in place before you run any of the following cell. 

In [10]:
ls

__MACOSX/  md  README.md  SparkProject.ipynb


In [11]:
cd data

/home/jovyan/SparkProject/data


In [6]:
ls

MLE_Challenge.md  MLE_Challenge.pdf  movies.dat  ratings.dat


### Begin of the project.

In [7]:
cd ..

/home/jovyan/SparkProject


Load libraries

In [8]:
import pyspark
from pyspark import SparkContext
from pyspark import SparkFiles
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql import SparkSession




In [2]:
sc =SparkContext()

#### Load the movie file. 

In [13]:
cd ..

/home/jovyan/SparkProject


In [14]:
file_location = 'data/'
movies_filename = 'movies.dat'

moviews_header = ['MovieID','Title','Genres']
spark = SparkSession(sc)
allGenres = set()

moviews_file = sc.textFile(file_location+movies_filename)
moviews_df = moviews_file.map(lambda r : r.split('::'))

moviews_df = moviews_df.toDF(moviews_header)
moviews_df.show()


for row in moviews_df.collect():
    allGenres = allGenres.union(set(row[2].split('|')))
allGenres = list(allGenres)

genres_dict = dict()

for i in range(len(allGenres)):
    genres_dict[allGenres[i]] = i

def genres_one_off_encoding(rdd):
    genres = rdd['Genres'].split('|')
    print(genres)
    encoding = ['0']*len(allGenres)
    for genre in genres:
        encoding[genres_dict[genre]] = '1'
    
    return Row(rdd['MovieID'], rdd['Title'], ''.join(encoding))

moviews_df = moviews_df.rdd.map(genres_one_off_encoding)
moviews_df = moviews_df.toDF(moviews_header)
moviews_df.show()



+-------+--------------------+--------------------+
|MovieID|               Title|              Genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|  Animation|Children|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [15]:
file_location = 'data/'
ratings_filename ='ratings.dat'

ratings_header = ['UserID','MovieID','Rating','Timestamp']

ratings_file = sc.textFile(file_location+ratings_filename)
ratings_df = ratings_file.map(lambda r : r.split('::'))

ratings_df.take(1)

ratings_df = ratings_df.toDF(ratings_header)
ratings_df.show()

# ratings_df.show()

+------+-------+----------+---------+
|UserID|MovieID|    Rating|Timestamp|
+------+-------+----------+---------+
|     1|    122|         5|838985046|
|     1|    185|         5|838983525|
|     1|    231|         5|838983392|
|     1|    292|         5|838983421|
|     1|    316|         5|838983392|
|     1|    329|         5|838983392|
|     1|    355|         5|838984474|
|     1|    356|         5|838983653|
|     1|    362|         5|838984885|
|     1|    364|         5|838983707|
|     1|    370|         5|838984596|
|     1|    377|         5|838983834|
|     1|    420|         5|838983834|
|     1|    466|         5|838984679|
|     1|    466|9999999999|838984679|
|     1|    480|         5|838983653|
|     1|    520|         5|838984679|
|     1|    539|         5|838984068|
|     1|    586|         5|838984068|
|     1|    588|         5|838983339|
+------+-------+----------+---------+
only showing top 20 rows



In [16]:
print(ratings_df.count())

ratings_df = ratings_df.where("Rating<=5")
print(ratings_df.count())
ratings_df = ratings_df.where("Rating>=0")
print(ratings_df.count())

10000055
10000044
10000035


#### Question 1: Top 5 movies. 

In [75]:
ratings_df.groupBy('UserID', 'MoviewID').count()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: functions does not exist in the JVM